# Objective

Create an average of cumulative fSCA (using the 40 mm daily threshold).

In [1]:
import pandas as pd
import xarray as xr
import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
import time

from myutils import metaread_veg 
from dirs import coorddir, prepdir

In [2]:

domain = "d02"

lat_wrf, lon_wrf, z_wrf, vgtyp_wrf, vegfra_wrf = metaread_veg(coorddir, domain)

lat_wrf = xr.DataArray(lat_wrf, dims=["lat2d", "lon2d"])
lon_wrf = xr.DataArray(lon_wrf, dims=["lat2d", "lon2d"])
z_wrf = xr.DataArray(z_wrf, dims=["lat2d", "lon2d"])
vgtyp_wrf = xr.DataArray(vgtyp_wrf, dims=["lat2d", "lon2d"]) 
vegfra_wrf = xr.DataArray(vegfra_wrf, dims=["lat2d", "lon2d"])

## Compute slope and aspect
myslopx, myslopy = np.gradient(z_wrf, 9000)
slope_wrf = np.degrees(np.arctan(np.sqrt(myslopx**2 + myslopy**2)))
aspect_wrf = np.degrees(np.arctan2(-myslopy,myslopx))
## Convert aspect to compass direction (clockwise from north)
aspect_q2 = (aspect_wrf > 90) & (aspect_wrf <= 180) ## [90, 180]
aspect_wrf = 90.0 - aspect_wrf
aspect_wrf[aspect_q2] = 360.0 + aspect_wrf[aspect_q2]


coords = {'lat': (['lat2d','lon2d'], lat_wrf), \
                 'lon': (['lat2d','lon2d'], lon_wrf) }

gcms = ['cesm2','mpi-esm1-2-lr','cnrm-esm2-1',
        'ec-earth3-veg','fgoals-g3','ukesm1-0-ll',
        'canesm5','access-cm2','ec-earth3',]


variants = ['r11i1p1f1','r7i1p1f1','r1i1p1f2',
            'r1i1p1f1','r1i1p1f1','r2i1p1f2',
            'r1i1p2f1','r5i1p1f1','r1i1p1f1',]
   

calendar = ['365_day','proleptic_gregorian','proleptic_gregorian',
            'proleptic_gregorian','365_day','360_day',
             '365_day','proleptic_gregorian', 'proleptic_gregorian',]


ssps = ['ssp370','ssp370','ssp370','ssp370',
        'ssp370','ssp370','ssp370','ssp370',
        'ssp370']

## Update these while running on nersc
wrf_dir = '/global/cfs/cdirs/m4099/fate-of-snotel/wrfdata/'


In [3]:
sweBC = xr.open_dataarray('/global/cfs/cdirs/m4099/fate-of-snotel/wrfdata/snowmaxBC.nc')

In [4]:
def get_gcm_filepath (gcm_ind, var, year):
    if year < 2014:
        return f'{wrf_dir}{gcms[gcm_ind]}_{variants[gcm_ind]}_historical_bc/postprocess/d02/' + f'{var}.daily.{gcms[gcm_ind]}_hist_BIAS_CORRECT_{variants[gcm_ind]}_d02_{str(year)}.nc'
    else:
        return f'{wrf_dir}{gcms[gcm_ind]}_{variants[gcm_ind]}_ssp370_bc/postprocess/d02/' + f'{var}.daily.{gcms[gcm_ind]}_ssp370_BIAS_CORRECT_{variants[gcm_ind]}_d02_{str(year)}.nc'

In [5]:
def get_date_list (gcm_ind, year):
    start_date = f'{year}1001'
    end_date = f'{year+1}0401'

    date_range = pd.date_range(start=start_date, end=end_date).strftime('%Y%m%d')
    date_list = list(date_range)

    if calendar[gcm_ind] == '365_day':
        ## remove Feb-29 for leap year
        if ((year+1) % 4 == 0) & ((year+1) != 2100):
            date_list.remove(f'{year+1}0229')
    elif calendar[gcm_ind] == '360_day':
        date_list.remove(f'{year}1031')
        date_list.remove(f'{year}1231')
        date_list.remove(f'{year+1}0131')
        date_list.remove(f'{year+1}0331')
        date_list.append(f'{year+1}0230')
        if ((year+1) % 4 > 0) | ((year+1) == 2100):
            date_list.append(f'{year+1}0229')
        date_list.sort()
    return date_list

In [6]:
def get_gcm_metvar (gcm_ind, year, date_list):
    
    var_name='snow'
    fname = get_gcm_filepath(gcm_ind, var_name, year)
    gcm_dataset = xr.open_dataset(fname)

    mean_fSCA = (gcm_dataset.sel(day=date_list)['snow'] > 40.0).mean(dim='day')
    return mean_fSCA

In [32]:
mean_fSCA_all = xr.zeros_like(sweBC)

In [36]:
start = time.time()

for gcm_ind in range(9):
    gcm = f'{gcms[gcm_ind]}_{variants[gcm_ind]}_ssp370'
    
    for year in range(1980,2100):

        date_list = get_date_list(gcm_ind, year)
        mean_fSCA = get_gcm_metvar (gcm_ind, year, date_list)

        mean_fSCA_all.loc[dict(gcm=gcm, time=year+1)] = mean_fSCA

end = time.time()
print (f'Time: {end-start} seconds')

mean_fSCA_all.attrs['long_name'] = 'cum. fSCA'
mean_fSCA_all.attrs['units'] = 'fraction (dimensionless)'
mean_fSCA_all.to_netcdf(f'{prepdir}/mean_fSCABC.nc')

Time: 345.32724356651306 seconds


# Corrupt fSCA using synthetic error

In [8]:
mean_fSCA_all_corrupted = xr.zeros_like(sweBC)
mean_fSCA_all_corrupted.attrs['long_name'] = 'cum. fSCA (corrupted)'
mean_fSCA_all_corrupted.attrs['units'] = 'fraction (dimensionless)'

In [10]:
start = time.time()


var_name='snow'
for gcm_ind in range(9):
    gcm = f'{gcms[gcm_ind]}_{variants[gcm_ind]}_ssp370'
    rng = np.random.default_rng(seed=0) ## This way, the masked days will be consistent across gcm's

    for year in range(1980,2100):
        date_list = get_date_list(gcm_ind, year)
        fname = get_gcm_filepath(gcm_ind, var_name, year)
        gcm_dataset = xr.open_dataset(fname)
        
        this_swe_clean = gcm_dataset.sel(day=date_list)['snow']
        
        ## Obtain a realization of the mask for all dates, across all pixels
        cloud_mask = rng.binomial(1, 2/3, (len(date_list), *lon_wrf.shape))
        this_swe_masked = this_swe_clean.where(cloud_mask)
        this_swe_masked[0] = this_swe_masked[0].fillna(0)
        this_swe_masked = this_swe_masked.ffill(dim='day')
        this_fSCA_corrupted = (this_swe_masked > 40.0).mean(dim='day')
        mean_fSCA_all_corrupted.loc[dict(gcm=gcm, time=year+1)] = this_fSCA_corrupted
            
end = time.time()
print (f'Time: {end-start} seconds')     
mean_fSCA_all_corrupted.to_netcdf(f'{prepdir}/mean_fSCABC_SynthErr.nc')

Time: 735.0030565261841 seconds
